## 코딩 전에.. 개념 적립!!



### Crawling vs Scraping

- **Crawling (혹은 Web Crawling)** : search engine(혹은 bot)이 웹페이지 내 링크들을 타고 다른 페이지들을 들어가는 것. (우리가 원하는 정보를 찾을 때까지 링크를 타고 들어가는 거겠지!)
- **Scraping** : 내가 원하는 정보를 수집하는 행위. (꼭 web에서 이루어질 필요 없음)

(ref: https://smartproxy.com/what-is-web-scraping/crawling-vs-scraping)

## 그렇다면.. 이제 코딩 시작!

In [ ]:
import requests
from bs4 import BeautifulSoup

- `request`는 url을 활용해 html 소스를 가져오는 역할을 함. <br>
(html 소스: 인터넷 페이지는 모두 코드로 이루어져 있는데, 그 코드를 뜻함)
- `beutifulsoup4`는 html 소스에서 필요한 데이터를 추출하는 함수를 제공해줌. <br>

<br>

예를 들어, https://goldkim92.github.io/online-cv/ 의 html 소스를 살펴보는 방법 2가지를 보자!!
- 인터넷 사이트에 들어가서 `F12`를 눌러보자.
- 밑에 코드를 돌려보자.

두 결과가 같음을 확인할 수 있다!!

In [ ]:
url = 'https://goldkim92.github.io/online-cv/'
html = requests.get(url)
print(html.text)

### html 소스를 볼 때 알아두면 유용한 용어 4가지만 소개해볼게! ###
- **`tag`** : html 소스를 보면 꺽쇠 표시들이 많이 보이지! `< >` 요런거. 이걸 `tag`라고 해. <br>
ex. < html >, < head >, < body >, < meta > ...
- **`element`** : 시작 tag 부터 끝 tag 까지의 구성
- **`attrubute`** : 시작 tag 안에 구체적인 속성을 정의
- **`value`** : attribute의 값

![](./html_basic.png)

reference 보면 더 잘 이해가 될거야! <br>
(ref: https://miaow-miaow.tistory.com/21)


### 이제 예시로 data를 추출해보자! ###
- 다시, 앞의 html 소스를 보면 `<head>` 가 보일 거야. 우리는 head tag 안에(nested) 있는 meta tag 를 모두 모아서 뽑아보고,
- 각 meta tag 에서 content attribute의 value를 뽑아볼게.

In [ ]:
soup = BeautifulSoup(html.text, 'html.parser')
metas = soup.head.find_all('meta')
metas

In [ ]:
for meta in metas:
    print(meta.get('content'))

- html 소스의 h2 tag를 모두 뽑아보고 싶다면,

In [ ]:
soup.find_all('h2')

### 그렇다면, 재명이의 publication/preprint의 제목 리스트를 뽑아볼까?! ###

In [ ]:
soup.find_all(attrs={'class':'job-title'})

In [ ]:
for element in soup.find_all(attrs={'class':'job-title'}):
    print(element.get_text())

## 이제는 실전이다! ##
### 당근 마켓의 인기 메물들의 이름/장소/가격/관심/채팅개수 를 scraping 해보자

- 그 전에! 먼저 데이터들을 저장할 excel파일을 우선적으로 만들어 놓자. 

In [ ]:
import csv

In [ ]:
with open('carrot-market-daily-best.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['Title', 'Region', 'Price', 'n_Love', 'n_Chat'])
    writer.writeheader()

- 이번엔 https://www.daangn.com/hot_articles 여기에 있는 당근마켓 메물들을 긁어모아보자구

In [ ]:
url = 'https://www.daangn.com/hot_articles'
html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')

In [ ]:
cards = soup.find_all(attrs={'class':'card-desc'})
len(cards)

In [ ]:
for card in cards:
    # title
    title = card.find(attrs={'class':'card-title'}).get_text()
    
    # region
    region = card.find(attrs={'class':'card-region-name'}).get_text()
    region = ' '.join(region.split())
    
    # price
    price = card.find(attrs={'class':'card-price'}).get_text()
    price = ' '.join(price.split())
    
    # n_love & n_chat
    counts = card.find(attrs={'class':'card-counts'}).get_text()
    n_love = counts.split()[1]
    n_chat = counts.split()[4]
    
    # save in excel file
    with open('carrot-market-daily-best.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([title, region, price, n_love, n_chat])